In [ ]:
# -- Install and setup Oracle Instant Client --
! wget -O oracleinstantclient.zip https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
! unzip -oq oracleinstantclient.zip
! mkdir -p /opt/oracle
! mv instantclient_19_6 /opt/oracle/
! sh -c "echo /opt/oracle/instantclient_19_6 > /etc/ld.so.conf.d/oracle-instantclient.conf"
! ln -s /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.0.so
! ldconfig


# -- Include cx_Oracle package --
! pip install cx_Oracle


--2022-04-29 09:12:51--  https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
Resolving download.oracle.com (download.oracle.com)... 23.3.84.100
Connecting to download.oracle.com (download.oracle.com)|23.3.84.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75462547 (72M) [application/zip]
Saving to: ‘oracleinstantclient.zip’

oracleinstantclient 100%[===================>]  71.97M   151MB/s    in 0.5s    

2022-04-29 09:12:52 (151 MB/s) - ‘oracleinstantclient.zip’ saved [75462547/75462547]

     |████████████████████████████████| 852 kB 8.7 MB/s 


In [ ]:
# Import cx_Oracle package and connect to Oracle Database
import cx_Oracle

HOST_NAME = "imz409.ust.hk"
PORT_NUMBER = "1521"
SERVICE_NAME = "imz409"
USERNAME = "T105"
PASSWORD = "WX932"

dsn_tns = cx_Oracle.makedsn(HOST_NAME, PORT_NUMBER, service_name=SERVICE_NAME) 
conn = cx_Oracle.connect(user=USERNAME, password=PASSWORD, dsn=dsn_tns) 

c = conn.cursor()

In [ ]:
from google.colab.widgets import TabBar
from IPython.core.display import clear_output
from contextlib import nullcontext
import ipywidgets as widgets
import time
import pandas as pd
from ipywidgets import interact, Text, DatePicker, IntText, Label, Layout, Button, Box, GridspecLayout, Checkbox, Output
from PIL import Image
import urllib.request
from datetime import date, datetime
import os
import cv2

In [ ]:
# creates tabs
def start():
  tb_all = TabBar(['Home Page', 'Room Selection','Login','Checkout','Sign Up','Sign Up Success','Payment','Success','Size Chart','Booking Preview'])

  # a simple function for jumping between tabs
  def showPage(tabName):
    with tb_all.output_to(tabName):
      display()


  #######################################################
  ################ GO TO: Home Page Tab #################
  #######################################################
  with tb_all.output_to('Home Page'):
    global check_in_datepicker
    global check_out_datepicker
    ######################### LABELS ####################
    home_label = Label ("Home Page")
    type_label = Label("Who do you want to book a room for?")                      # pet type
    no_of_pets_label = Label("How many pets are you booking for?")                 # no of pets
    size_label = Label("Please select the 1st pet's size: ")                       # size of pet 1
    size_label2 = Label("Please select the 2nd pet's size: ")                      # size of pet 2
    size_label3 = Label("Please select the 3rd pet's size: ")                      # size of pet 3
    check_in_label = Label("Please select the Check-In date: ")                    # check-in date
    check_out_label = Label("Please select the Check-Out date: ")                  # check-out date 
    empty_label = Label("")

    ####################### INPUT BOXES ##################
    rbGroup = widgets.RadioButtons(options=["Dog(s)", "Cat(s)"], value=None)       # pet type
    cmb = widgets.Dropdown(options = (0, 1, 2, 3), value = 0)                      # no of pets
    cmb1 = widgets.Dropdown(options= ("Small", "Medium", "Large"), value=None)     # size of pet 1
    cmb2 = widgets.Dropdown(options= ("Small", "Medium", "Large"), value=None)     # size of pet 2
    cmb3 = widgets.Dropdown(options= ("Small", "Medium", "Large"), value=None)     # size of pet 3
    check_in_datepicker = DatePicker(disabled=False)      # check-in date
    check_out_datepicker = DatePicker(disabled=False)    # check-out date


    ##################### BUTTONS ########################
    # btnChoose = widgets.Button(description="Choose")
    btnSelect = widgets.Button(description = "Select")
    btnSizeChart = widgets.Button(description = "Show Size Chart")
    btnShowRooms = widgets.Button(description="Show Rooms!")
    btnClear = widgets.Button(description="Clear all")
    btnCheckPage = widgets.Button(description= "Check Booking")
    ############### PET SIZES DICTIONARY #################
    pet_sizes = {"Small": 1, "Medium": 2, "Large": 3}


    ############### SELECT BTN FUNCTION ##################
    def on_btnSelect_button_clicked(b):
      global no_of_pets 
      no_of_pets = cmb.value

      if no_of_pets == 1:
        grid[3,0] = widgets.HBox([size_label])
        grid[3,1] = widgets.HBox ([cmb1])
        grid[4,0] = empty_label
        grid[4,1] = empty_label
        grid[5,0] = empty_label
        grid[5,1] = empty_label

      if no_of_pets == 2:
        grid[3,0] = widgets.HBox([size_label])
        grid[3,1] = widgets.HBox ([cmb1])
        grid[4,0] = widgets.HBox([size_label2])
        grid[4,1] = widgets.HBox ([cmb2])
        grid[5,0] = empty_label
        grid[5,1] = empty_label
        
      if no_of_pets == 3:
        grid[3,0] = widgets.HBox([size_label])
        grid[3,1] = widgets.HBox ([cmb1])
        grid[4,0] = widgets.HBox([size_label2])
        grid[4,1] = widgets.HBox ([cmb2])
        grid[5,0] = widgets.HBox([size_label3])
        grid[5,1] = widgets.HBox ([cmb3])

    btnSelect.on_click(on_btnSelect_button_clicked)


    ################ SIZE CHART BUTTON FUNCTION #######################
    def on_btnSizeChart_clicked(b):
      showPage ("Size Chart")
    btnSizeChart.on_click(on_btnSizeChart_clicked)
    ################ CHECK BOOKING BUTTON FUNCTION ####################
    def on_btnCheckPage_clicked(b):
      showPage("Booking Preview")
    btnCheckPage.on_click(on_btnCheckPage_clicked)

    ################ SHOW ROOMS BUTTON FUNCTION #######################
    def on_btnShowRooms_button_clicked(b):
      global pet_type 
      global no_of_pets 
      global size
      global check_in
      global check_out
      global size_str
      global pet_sizes_list
      global nights 
      global flag2

      ######## PET TYPE #################
      if rbGroup.value == "Dog(s)":
        pet_type = "Dog"
      elif rbGroup.value == "Cat(s)":
        pet_type = "Cat"

      ##### PET SIZE(S) STRING LIST ######
      pet_sizes_list = [cmb1.value, cmb2.value, cmb3.value]
      size_str = "Size(s) selected: "
      for i in pet_sizes_list:
        if i != None:
          size_str = size_str + i + ", "

      ######## INPUT VALIDATION ############
      flag2 = False

      # if pet type is empty
      if rbGroup.value == None:
        grid[1,3] = Label("Please select pet type")
        flag2 = True
      else: 
        grid[1,3] = Label("")

      # if no of pets is empty or 0
      if cmb.value == 0:
        grid[2,3] = Label("Please select no. of pets")
        flag2 = True
      else: 
        grid[2,3] = Label("")

      # if pet sizes are empty
      if no_of_pets == 1:
        if cmb1.value == None:
          grid[3,3] = Label("Please select pet size")
          flag2 = True
        else: 
          grid[3,3] = Label("")

      if no_of_pets == 2:
        if cmb1.value == None:
          grid[3,3] = Label("Please select pet size")
          flag2 = True
        else: 
          grid[3,3] = Label("")

        if cmb2.value == None:
          grid[4,3] = Label("Please select pet size")
          flag2 = True
        else: 
          grid[4,3] = Label("")
      
      if no_of_pets == 3:
        if cmb1.value == None:
          grid[3,3] = Label("Please select pet size")
          flag2 = True
        else: 
          grid[3,3] = Label("")

        if cmb2.value == None:
          grid[4,3] = Label("Please select pet size")
          flag2 = True
        else: 
          grid[4,3] = Label("")
      
        if cmb3.value == None:
          grid[5,3] = Label("Please select pet size")
          flag2 = True
        else: 
          grid[5,3] = Label("")
      
      # if check in date is empty
      if check_in_datepicker.value == None:
        grid[7,3] = Label("Please select check in date")
        flag2 = True
      # if check in date has past (< today)
      elif check_in_datepicker.value < date.today():
        grid[7,3] = Label("Please choose another date")
        flag2 = True
      else: 
          grid[7,3] = Label("")

      # if check out date is empty
      if check_out_datepicker.value == None:
        grid[8,3] = Label("Please select check out date")
        flag2 = True
      # if check out date is earlier than the check in date
      elif check_out_datepicker.value <= check_in_datepicker.value:
        grid[8,3] = Label("Please choose another date")
        flag2 = True
      else: 
          grid[8,3] = Label("")
      
      ######## IF THERE ARE NO ERRORS: ############
      if not flag2:
        # save no of pets 
        if no_of_pets == 1:
          size = pet_sizes[cmb1.value]
        elif no_of_pets == 2:
          size = pet_sizes[cmb1.value] + pet_sizes[cmb2.value]
        elif no_of_pets == 3:
          size = pet_sizes[cmb1.value] + pet_sizes[cmb2.value] + pet_sizes[cmb3.value]

        # save check_in, check_out, and no of nights
        check_in = str(check_in_datepicker.value)                                 # RES_CHECK_IN
        check_out = str(check_out_datepicker.value)                               # RES_CHECK_OUT

        b =  datetime.strptime(check_out, "%Y-%m-%d")
        a =  datetime.strptime(check_in, "%Y-%m-%d")
        
        delta = b - a
        nights = delta.days                                                       # RES_NIGHTS

        #move to "Room Selection" page
        showPage("Room Selection")

    # perform the function "on_btnShowRooms_button_clicked" when "Show Rooms" button is clicked
    btnShowRooms.on_click(on_btnShowRooms_button_clicked)

    ############ CLEAR BUTTON FUNCTION ######################
    def on_btnClear_button_clicked(b):
      clear_output()
      start()

    btnClear.on_click(on_btnClear_button_clicked)

    ############################ GRID ###############################
    grid = GridspecLayout(12, 4, height='600px', width='1000px', grid_gap='2px')

    grid[0,1] = widgets.HBox([home_label])
    grid[1,0] = widgets.HBox([type_label])
    grid[1,1] = widgets.HBox([rbGroup])
    grid[2,0] = widgets.HBox([no_of_pets_label])
    grid[2,1] = widgets.HBox([cmb])
    grid[2,2] = widgets.HBox([btnSelect])
    grid[6,0] = widgets.HBox ([btnSizeChart])
    grid[7,0] = widgets.HBox ([check_in_label])
    grid[7,1] = widgets.HBox ([check_in_datepicker])
    grid[8,0] = widgets.HBox ([check_out_label])
    grid[8,1] = widgets.HBox ([check_out_datepicker])
    grid[9,1] = widgets.HBox ([btnShowRooms])
    grid[10,1] = widgets.HBox ([btnCheckPage])
    grid[11,1] = widgets.HBox ([btnClear])

    hbRow1_layout = widgets.Layout(display='center', flex_flow='col', 
                                  justify_content='center', align_items='center', 
                                  border='1px solid grey', height='35px')
    display(grid)

  
  #######################################################
  ############# GO TO: Room Selection Tab  ##############
  #######################################################

  with tb_all.output_to('Room Selection'):

    def show_room_selection(b):
      # if there is an error in previous page, don't perform the function
      if flag2 == True:
        return
      
      # save variables as global
      global booking_type_id
      booking_type_id = ""
      global type_button
      global type_button2
      global room_id
      global flag
      
      #################### Show Customers' Input ##################

      page_label = Label ("Room Selection")
      check_in_date_label = Label ("Check-in: " + check_in)
      check_out_date_label = Label ("Check-out: " + check_out)
      no_of_nights_label1 = Label ("No of Nights: "+ str(nights))
      type_label2 = Label ("Pet type: " + pet_type)
      no_of_pets_label2 = Label ("Number of Pets: " + str(no_of_pets))
      pet_sizes_label = Label (size_str[0:-2])
      

      grid2[1,0] = widgets.HBox([check_in_date_label])
      grid2[1,2] = widgets.HBox([type_label2])
      grid2[2,0] = widgets.HBox([check_out_date_label])
      grid2[2,2] = widgets.HBox([no_of_pets_label2])
      grid2[3,0] = widgets.HBox([no_of_nights_label1])
      grid2[3,2] = widgets.HBox([pet_sizes_label])
      grid2[4,1] = widgets.HBox ([btnEdit])

      # get the number of room types
      c.execute("SELECT COUNT(TYP_ID) FROM ROOM_TYPE")

      for row in c:
        no_of_room_types = row[0]
      
      # start Room Type Name from Row 5
      name_grid_no = 5
      # start Room Type Price from Row 6
      price_grid_no = 6
      # start Room Type Price from Row 7
      desc_grid_no = 7
      # ?
      type_description_label = ""
  
      i = 1
      while i < int(no_of_room_types)+1:
        flag = True
        type_button_desc = ""
        type_id2 = ""
        room_type_name = ""
        
        room_type_id = "TYP"+ str(i)

        #################### Room Type Name ##################
        # SQL: get the type names
        c.execute("SELECT TYP_NAME FROM ROOM_TYPE WHERE TYP_ID = '" +room_type_id+"' AND TYP_SPECIES = '" +pet_type+"'")
        for row in c:
          room_type_name = row[0]

        # save type name as variable and put it onto a grid, skip if the species is wrong
        if room_type_name:
          type_name_label = Label(room_type_name)

          grid2[name_grid_no,0:1] = widgets.HBox ([type_name_label])

        else:
          i = i + 1
          continue
        
        #################### Room Type Price ##################
        # SQL: get the type names
        c.execute("SELECT TYP_PRICE FROM ROOM_TYPE WHERE TYP_ID = '" +room_type_id+"' AND TYP_SPECIES = '" +pet_type+"'")

        for row in c:
          room_type_price = row[0]

        
        # save price as variable and put it onto a grid, skip if the species is wrong
        type_price_label = Label("Room Rate: $"+str(room_type_price))
        grid2[price_grid_no,0:2] = widgets.HBox ([type_price_label])
      
        #################### Room Type Amenities ##################
        # SQL: get the type names
        c.execute("SELECT TYP_AMENITIES FROM ROOM_TYPE WHERE TYP_ID = '" +room_type_id+"' AND TYP_SPECIES = '" +pet_type+"'")

        for row in c:
          room_type_desc = row[0]
    
        # save amenities as variable and put it onto a grid, skip if the species is wrong
        grid2[desc_grid_no:desc_grid_no+2,0:2] = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ room_type_desc +' </p>')

        #################### "Cannot Book / Book" Button ##################

        if ("Small" in pet_sizes_list) or ("Medium" in pet_sizes_list):
            max_size = 2;
        
        if ("Large" in pet_sizes_list) or (pet_type == "Cat"):
            max_size = 3;
    
        # SQL: get TYP_ID and ROM_ID where TYP_ID = the one in the loop & RES_CHECK_IN, RES_CHECK_OUT, TYP_MAX_SIZE, TYP_SPECIES and TYP_NO according to customers' filters
        c.execute ("""SELECT TYP_ID, ROM_ID, TYP_AMENITIES, TYP_NAME, TYP_PRICE  FROM RESERVATION 
        FULL OUTER JOIN CONTAINS ON contains.reservation_res_id = reservation.res_id 
        FULL OUTER JOIN ROOM ON room.rom_id = contains.room_rom_id 
        FULL OUTER JOIN ROOM_TYPE ON room_type.typ_id = room.room_type_typ_id 
        WHERE (TYP_ID ='{}') AND (RES_CHECK_IN >= TO_DATE('{}', 'YYYY-MM-DD') OR RES_CHECK_IN <= TO_DATE('{}', 'YYYY-MM-DD') OR (RES_CHECK_IN IS NULL)) 
        AND (RES_CHECK_OUT <= TO_DATE('{}', 'YYYY-MM-DD') OR RES_CHECK_OUT IS NULL) AND (TYP_MAX_SIZE = {}) 
        AND (TYP_SPECIES = '{}') AND (TYP_NO = {}) """
        .format(room_type_id, check_out_datepicker.value.strftime("%Y-%m-%d"), check_in_datepicker.value.strftime("%Y-%m-%d"), check_in_datepicker.value.strftime("%Y-%m-%d"), max_size, pet_type, no_of_pets))

        # save available rooms as variables
        for row in c:
          type_id2 = row[0]
          room_id = row[1]
          type_amenities = row[2]
          type_name = row[3]
          type_price = row[4]

        # if there is available rooms and the type matches, show "Book"
        if type_id2 == room_type_id:
            # create a "Book" button
            type_button = widgets.Button(description = "Book")

            # place the button onto the grid
            grid2[name_grid_no,2] = widgets.HBox ([type_button])

            def on_type_button_clicked(b):
              global confirmed_room_id
              global confirmed_type_id
              global confirmed_type_amenities
              global confirmed_type_name
              global confirmed_type_price
              global temporary_subtotal
              global final_subtotal

              # save the chosen room
              confirmed_room_id = room_id                                         # ROM_ID
              confirmed_type_id = type_id2                                        # TYP_ID                                       
              confirmed_type_amenities = type_amenities
              confirmed_type_name = type_name                                     # TYP_SPECIES and PET_SPECIES
              confirmed_type_price = type_price
              temporary_subtotal = confirmed_type_price * nights
              final_subtotal = temporary_subtotal
              showPage("Login")

            type_button.on_click(on_type_button_clicked)
            type_button.on_click(show_booking_preview2)
            type_button.on_click(show_booking_preview3)
          
        # if there isn't any available rooms (or if the room type doesn't match), show "Cannot Book"
        else:
            type_button2 = widgets.Button(description = "Cannot Book")
            type_button2_desc = "Cannot Book"
    
            grid2[name_grid_no,2] = widgets.HBox ([type_button2])

        name_grid_no = name_grid_no + 6
        price_grid_no = price_grid_no + 6
        desc_grid_no = desc_grid_no + 6
        i = i+1

    btnShowRooms.on_click(show_room_selection)
      

    ##################### BUTTONS ########################
    btnEdit = widgets.Button(description = "Edit")

    # edit information button
    def on_btnEdit_button_clicked(b):
      clear_output()
      start()

    btnEdit.on_click(on_btnEdit_button_clicked)

    ############################ GRID ##############################
    grid2= GridspecLayout(80, 3, height='3000px', width='80%', grid_gap='1px')
    display(grid2)

  #######################################################
  ################ GO TO: Login Tab #################
  #######################################################
  with tb_all.output_to('Login'):

    ############################ Formatting ##############################
    button_center = widgets.Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    width='30%')

    credentials_layout = widgets.Layout(width='400px')

    ############################ Back Button ##############################
    back_login = widgets.Button(description = "Edit/Cancel Order", layout=button_center)

    def on_back_login_clicked(b):
      clear_output()
      start()
    
    back_login.on_click(on_back_login_clicked)

    ############################ Login ##############################
    login_label = Label("Login")
    or_label = Label("or", layout=button_center)

    member_email = widgets.Text(                                                
        value = '',
        placeholder='Insert email',
        description='Email :',
        layout = credentials_layout
    )
    member_password = widgets.Password(                                                                            
        value = '',
        placeholder = 'Enter password',
        description = 'Password :',
        layout = credentials_layout
    )
    login_button = widgets.Button(                                                
        description = 'Login',
        button_style = '',
        layout=button_center
    ) 

    def login_button_clicked(i):
      global confirmed_name
      global confirmed_id

      input = "SELECT CUS_EMAIL, CUS_PASSWORD FROM CUSTOMER WHERE CUS_EMAIL = '" + member_email.value + "' AND CUS_PASSWORD = '" + member_password.value + "'"
      validation = False
      for row in c.execute(input):
        validation = True

      if len(member_email.value) == 0 and len(member_password.value) == 0:
        grid_login[9,2] = Label("Please fill in email and password")
      
      elif len(member_email.value) == 0 and len(member_password.value) != 0:
        grid_login[9,2] = Label("Please fill in email")
      
      elif len(member_email.value) != 0 and len(member_password.value) == 0:
        grid_login[9,2] = Label("Please fill in password")

      elif validation:
        grid_login[9,2] = Label("")
        c.execute ("SELECT CUS_NAME, CUS_ID FROM CUSTOMER WHERE CUS_EMAIL = '{}' AND CUS_PASSWORD = '{}'".format(member_email.value, member_password.value))

        for row in c:
          confirmed_name = row[0]
          confirmed_id = row[1]                                                   # CUS_ID

        login_button.on_click(show_booking_preview5)
        showPage('Checkout')
      else:
        grid_login[9,2] = Label("Incorrect email or password, please reenter")

    login_button.on_click(login_button_clicked)

    ############################ Sign Up ##############################
    signup_button = widgets.Button(description = "Sign Up", layout=button_center)

    def on_signup_button_clicked(b):
      showPage("Sign Up")
    
    signup_button.on_click(on_signup_button_clicked)

    ############################ Booking Preview (1/3) ##############################
    def show_booking_preview2(b):
      grid_login[1,0:2] = widgets.HBox ([Label ("Your order is almost complete! (1/3)")])
      grid_login[2,0:2] = widgets.HBox ([Label ("Check-in: " + check_in)])
      grid_login[3,0:2] = widgets.HBox ([Label ("Check-out: " + check_out)])
      grid_login[4,0:2] = widgets.HBox ([Label ("No of Nights: " + str(nights))])
      grid_login[5,0:2] = widgets.HBox ([Label ("No of Pets: " + str(no_of_pets))])
      grid_login[6,0:2] = widgets.HBox ([Label ("Room Type: " + confirmed_type_name)])
      grid_login[7:10,0:2] = widgets.HBox ([widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ "Room Description: " + confirmed_type_amenities +' </p>')])
      grid_login[11,0:2] = widgets.HBox ([Label ("Room Rate: $" + str(confirmed_type_price))])
      grid_login[12,0:2] = widgets.HBox ([Label ("Subtotal: $" + str(temporary_subtotal))])

    ############################ GRID ##############################
    grid_login = GridspecLayout(23, 3, height='1000px', width='100%', grid_gap='1px')
    grid_login[0,0] = widgets.HBox ([back_login])
    grid_login[1,2] = widgets.HBox ([login_label])
    grid_login[2,2] = widgets.HBox ([member_email])
    grid_login[3,2] = widgets.HBox ([member_password])
    grid_login[4,2] = widgets.HBox ([login_button])
    grid_login[5,2] = widgets.HBox ([or_label])
    grid_login[6,2] = widgets.HBox ([signup_button])
    display(grid_login)

  #######################################################
  ################ GO TO: Sign Up Tab #################
  #######################################################
  with tb_all.output_to('Sign Up'):

    ############################ Back Button ##############################
    back_signup = widgets.Button(description = "Back: Login", layout=button_center)

    def on_back_login_clicked(b):
      showPage("Login")
    
    back_signup.on_click(on_back_login_clicked)

    ############################ Sign Up ##############################
    signup_label = Label("Sign Up")

    signup_email = widgets.Text(                                                
        value = '',
        description='Email :',
        style = {'description_width': '120px'},
        layout = credentials_layout
    )
    signup_password = widgets.Password(                                                                            
        value = '',
        description = 'Password :',
        style = {'description_width': '120px'},
        layout = credentials_layout
    )
    signup_confirm_password = widgets.Password(                                                                            
        value = '',
        description = 'Confirm Password :',
        style= {'description_width': '120px'},
        layout = credentials_layout
    )
    signup_name = widgets.Text(                                                
        value = '',
        description='Name :',
        style = {'description_width': '120px'},
        layout = credentials_layout
    )
    signup_phone = widgets.IntText(                                                
        value = None,
        description='Phone Number :',
        style = {'description_width': '120px'},
        layout = credentials_layout
    )
    signup_member = widgets.Dropdown(
        options= ("Yes", "No"),
        description='Membership',
        style = {'description_width': '120px'},
        value = "Yes"
        ) 
    
    signup_button2 = widgets.Button(                                                
        description = 'Sign Up',
        button_style = '',
        layout=button_center
    ) 

    ############################ On Sign Up Clicked ##############################
    def signup_button2_clicked(i):
      global confirmed_name

      ################## Input Validation ######################
      
      c.execute("SELECT CUS_EMAIL FROM CUSTOMER")
      all_email = []
      for row in c:
          all_email.append(row)
      
      flag = False

      if len(signup_email.value) > 30:
        grid_signup[9,2] = Label("Email is too long")
        flag = True
      elif len(signup_email.value) == 0:
        grid_signup[9,2] = Label("Please fill in email address")
        flag = True
      elif "@" not in signup_email.value:
        grid_signup[9,2] = Label("Email is not in the correct format")
        flag = True
      elif signup_email.value in all_email:
        grid_signup[9,2] = Label("Email is already registered, please log in instead")
        flag = True

      
      if len(signup_password.value) > 16:
        grid_signup[9,2] = Label("Password is too long")
        flag = True
      elif len(signup_password.value) == 0:
        grid_signup[9,2] = Label("Please fill in password")
        flag = True


      if signup_password.value != signup_confirm_password.value:
        grid_signup[9,2] = Label("Please make sure your passwords match")
        flag = True


      if len(signup_name.value) > 30:
        grid_signup[9,2] = Label("Name is too long")
        flag = True
      elif len(signup_name.value) == 0:
        grid_signup[9,2] = Label("Please fill in name")
        flag = True
      
      if signup_phone.value > 99999999:
        grid_signup[9,2] = Label("Phone Number is too long")
        flag = True
      elif signup_phone.value < 10000000:
        grid_signup[9,2] = Label("Please fill in the correct phone number format")
        flag = True
      elif signup_phone.value == None:
        grid_signup[9,2] = Label("Please fill in phone number")
        flag = True

      # if no errors, create customer ID and insert to SQL
      if not flag:
        ################## Customer ID ######################
        c.execute("SELECT CUS_ID FROM CUSTOMER ORDER BY (CUS_ID) ASC"  )

        checker = 0

        for row in c:
          cus_id_no = int(row[0][3:]) + 1
          if checker < cus_id_no:
            checker = cus_id_no
      
          str_cus_id_no = str(checker)
          cus_id = "CUS" + str_cus_id_no
        
        ################## Insert Member Details to SQL ######################
        c.execute(""" INSERT INTO CUSTOMER (CUS_ID, CUS_NAME, CUS_PHONE, CUS_EMAIL, CUS_PASSWORD, CUS_STARS, CUS_MEMBERSHIP) VALUES ('{}', '{}', '{}', '{}', '{}', '0', '{}')"""
        .format(cus_id, signup_name.value, signup_phone.value, signup_email.value, signup_password.value, signup_member.value))

        conn.commit()

        ################# Move to "Sign Up Success" Page ######################
        showPage('Sign Up Success')

        ################# Save Variables for Future Usage ######################
        confirmed_name = signup_name.value

    signup_button2.on_click(signup_button2_clicked)

    ############################ Booking Preview (1/3) ##############################
    def show_booking_preview3(b):
      grid_signup[1,0:2] = widgets.HBox ([Label ("Your order is almost complete! (1/3)")])
      grid_signup[2,0:2] = widgets.HBox ([Label ("Check-in: " + check_in)])
      grid_signup[3,0:2] = widgets.HBox ([Label ("Check-out: " + check_out)])
      grid_signup[4,0:2] = widgets.HBox ([Label ("No of Nights: " + str(nights))])
      grid_signup[5,0:2] = widgets.HBox ([Label ("No of Pets: " + str(no_of_pets))])
      grid_signup[6,0:2] = widgets.HBox ([Label ("Room Type: " + confirmed_type_name)])
      grid_signup[7:10,0:2] = widgets.HBox ([widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ "Room Description: " + confirmed_type_amenities +' </p>')])
      grid_signup[11,0:2] = widgets.HBox ([Label ("Room Rate: $" + str(confirmed_type_price))])
      grid_signup[12,0:2] = widgets.HBox ([Label ("Subtotal: $" + str(temporary_subtotal))])

      grid_signupsuccess[1,0:2] = widgets.HBox ([Label ("Your order is almost complete! (1/3)")])
      grid_signupsuccess[2,0:2] = widgets.HBox ([Label ("Check-in: " + check_in)])
      grid_signupsuccess[3,0:2] = widgets.HBox ([Label ("Check-out: " + check_out)])
      grid_signupsuccess[4,0:2] = widgets.HBox ([Label ("No of Nights: " + str(nights))])
      grid_signupsuccess[5,0:2] = widgets.HBox ([Label ("No of Pets: " + str(no_of_pets))])
      grid_signupsuccess[6,0:2] = widgets.HBox ([Label ("Room Type: " + confirmed_type_name)])
      grid_signupsuccess[7:10,0:2] = widgets.HBox ([widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ "Room Description: " + confirmed_type_amenities +' </p>')])
      grid_signupsuccess[11,0:2] = widgets.HBox ([Label ("Room Rate: $" + str(confirmed_type_price))])
      grid_signupsuccess[12,0:2] = widgets.HBox ([Label ("Subtotal: $" + str(temporary_subtotal))])

      grid_checkoutmember[0,0] = widgets.HBox ([back_checkoutmember])
      grid_checkoutmember[2,0:2] = widgets.HBox ([Label ("Check-in: " + check_in)])
      grid_checkoutmember[3,0:2] = widgets.HBox ([Label ("Check-out: " + check_out)])
      grid_checkoutmember[4,0:2] = widgets.HBox ([Label ("No of Nights: " + str(nights))])
      grid_checkoutmember[5,0:2] = widgets.HBox ([Label ("No of Pets: " + str(no_of_pets))])
      grid_checkoutmember[6,0:2] = widgets.HBox ([Label ("Room Type: " + confirmed_type_name)])
      grid_checkoutmember[7:10,0:2] = widgets.HBox ([widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ "Room Description: " + confirmed_type_amenities +' </p>')])
      grid_checkoutmember[11,0:2] = widgets.HBox ([Label ("Room Rate: $" + str(confirmed_type_price))])

      grid_checkoutmember[15,0:2] = widgets.HBox ([Label ("$" + str(confirmed_type_price) + " HKD x " + str(nights) + " nights: $" + str(temporary_subtotal))])
      grid_checkoutmember[16,0:2] = widgets.HBox ([Label ("Subtotal: $" + str(temporary_subtotal))])

      grid_payment[2,0:2] = widgets.HBox ([Label ("Check-in: " + check_in)])
      grid_payment[3,0:2] = widgets.HBox ([Label ("Check-out: " + check_out)])
      grid_payment[4,0:2] = widgets.HBox ([Label ("No of Nights: " + str(nights))])
      grid_payment[5,0:2] = widgets.HBox ([Label ("No of Pets: " + str(no_of_pets))])
      grid_payment[6,0:2] = widgets.HBox ([Label ("Room Type: " + confirmed_type_name)])
      grid_payment[7:10,0:2] = widgets.HBox ([widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ "Room Description: " + confirmed_type_amenities +' </p>')])

    ############################ GRID ##############################
    grid_signup = GridspecLayout(23, 3, height='1000px', width='100%', grid_gap='1px')
    grid_signup[0,0] = widgets.HBox ([back_login])
    grid_signup[1,2] = widgets.HBox ([signup_label])
    grid_signup[2,2] = widgets.HBox ([signup_email])
    grid_signup[3,2] = widgets.HBox ([signup_password])
    grid_signup[4,2] = widgets.HBox ([signup_confirm_password])
    grid_signup[5,2] = widgets.HBox ([signup_name])
    grid_signup[6,2] = widgets.HBox ([signup_phone])
    grid_signup[7,2] = widgets.HBox ([signup_member])
    grid_signup[8,2] = widgets.HBox ([signup_button2])
    display(grid_signup)

  #######################################################
  ################ GO TO: Sign Up Success Tab #################
  #######################################################
  with tb_all.output_to('Sign Up Success'):

    ############################ Continue Button ##############################
    continue_button = widgets.Button(                                                
        description = 'Continue Booking',
        button_style = '',
        layout=button_center
    ) 

    def continue_button_clicked(b):
      # type_button.on_click(show_booking_preview5)
      showPage('Login')

    continue_button.on_click(continue_button_clicked)

    ############################ GRID ##############################
    grid_signupsuccess = GridspecLayout(23, 3, height='1000px', width='100%', grid_gap='1px')
    grid_signupsuccess[1,2] = widgets.HBox ([Label ("Sign up successful!")])
    grid_signupsuccess[2,2] = widgets.HBox ([continue_button])
    display(grid_signupsuccess)

  #######################################################
  ################ GO TO: Checkout Tab #################
  #######################################################
  with tb_all.output_to('Checkout'):

    ############################ Back Button ##############################
    back_checkoutmember = widgets.Button(description = "Edit/Cancel Order", layout=button_center)

    def on_back_checkoutmember_clicked(b):
      clear_output()
      start()

    
    back_checkoutmember.on_click(on_back_checkoutmember_clicked)

    ############################ Booking Preview (1/3) ##############################
    def show_booking_preview5(b):
      global order_stars
      global pet1_dropdown
      global pet2_dropdown
      global pet3_dropdown
      global new_pet1_name
      global new_pet2_name
      global new_pet3_name
      global member_stars
      global rew_dict
      global rewards_dropdown
      global mem_status
      global pet1_name
      global pet2_name
      global pet3_name

      new_pet1_name = widgets.Text(                                             
                description='New Pet Name:',
                style = {'description_width': '120px'},
                layout = credentials_layout)

      new_pet2_name = widgets.Text(                                       
                description='New Pet Name:',
                style = {'description_width': '120px'},
                layout = credentials_layout)
      
      new_pet3_name = widgets.Text(                                           
                description='New Pet Name:',
                style = {'description_width': '120px'},
                layout = credentials_layout)

      rew_dict = {None: ""}

      ############################# Reset Grids ################################
      grid_checkoutmember[3,2] = widgets.HBox ([])

      ########################## Member or No Member? ##########################
      c.execute ("SELECT CUS_MEMBERSHIP FROM CUSTOMER WHERE CUS_EMAIL = '{}' AND CUS_PASSWORD = '{}'".format(member_email.value, member_password.value))
      mem_status = False

      for row in c:
        if row[0] == "Yes":
          mem_status = True
        else:
          mem_status = False

      ############################ Member's Stars ##############################

      if mem_status == True:
        ############################# Claim Rewards ################################
        claim_rewards_label = Label('Claim Rewards (You can only pick one reward):')
        grid_checkoutmember[14,2] = widgets.HBox ([claim_rewards_label])

        c.execute ("SELECT CUS_STARS FROM CUSTOMER WHERE CUS_EMAIL = '{}' AND CUS_PASSWORD = '{}'".format(member_email.value, member_password.value))

        member_stars = 0
        for row in c:
          member_stars = row[0]

        grid_checkoutmember[15,2] = widgets.HBox ([Label("Your Stars: "+str(member_stars) + "☆")])
      
        ################### Stars Earned in the Current Order ####################
        order_stars = 0

        order_stars = round(0.1*(temporary_subtotal), 0)        

        grid_checkoutmember[12,0:2] = widgets.HBox ([Label ("Stars Earned: " + str(order_stars))])                      

        ################################# Reward #################################

        c.execute("SELECT COUNT(REWARD_ID) FROM REWARD")

        for row in c:
          no_of_rewards = row[0]

        c.execute ("SELECT REWARD_NAME, REWARD_PRICE FROM REWARD WHERE REWARD_PRICE <= {}". format(member_stars))
        i = 0
        for row in c:
          rew_dict[row[0]] = row[1]

        rewards_dropdown = widgets.Dropdown(options = rew_dict.keys(), description = "Choose a reward: ",
                                          style = {'description_width': '120px'},
                                          value = None)
        
        grid_checkoutmember[16,2] = widgets.HBox ([rewards_dropdown])
        
        
        # def on_choose_btn_clicked(b):
        def show_rew_price_qty(b):
          global member_name
          if rewards_dropdown.value != None:
            rew_price = rew_dict[rewards_dropdown.value]
            rewards_price_label = Label ("ㅤㅤㅤㅤㅤㅤㅤPrice: " + str(rew_price) + "☆")
            grid_checkoutmember[17,2] = widgets.HBox([rewards_price_label])
        
            c.execute("SELECT REWARD_QUANTITY FROM REWARD WHERE REWARD_NAME = '{}'".format (rewards_dropdown.value))
            max_rew_qty = 0
            for row in c:
              max_rew_qty = row[0]

            global rewards_qty_intbox

            rewards_qty_intbox = widgets.BoundedIntText (value = 0, min = 0, max = max_rew_qty, description = "Quantity: ", step = None, style = {'description_width': '120px'},)
            grid_checkoutmember[18,2] = widgets.HBox([rewards_qty_intbox])

          else:
            grid_checkoutmember[17,2] = widgets.HBox([empty_label])
            grid_checkoutmember[18,2] = widgets.HBox ([empty_label])  
            
        rewards_dropdown.observe(show_rew_price_qty, names='value')

      ############################ Update Grid (Left Side) ##############################

      c.execute ("SELECT CUS_NAME FROM CUSTOMER WHERE CUS_EMAIL = '{}' AND CUS_PASSWORD = '{}'".format (member_email.value, member_password.value))

      member_name = ""

      for row in c:
        member_name = row[0]

      c.execute ("SELECT CUS_ID FROM CUSTOMER WHERE CUS_EMAIL = '{}' AND CUS_PASSWORD = '{}'".format (member_email.value, member_password.value))

      member_id = ""

      for row in c:
        member_id = row[0]

      grid_checkoutmember[1,0:2] = widgets.HBox ([Label (member_name + ", your order is almost complete! (2/3)")])
      grid_payment[1,0:2] = widgets.HBox ([Label (member_name + ", your order is almost complete! (3/3)")])
    
      ############################ Pet 1 Details ##############################
      if no_of_pets >= 1:
        pet1_label = Label(pet_type + " 1 (" + pet_sizes_list[0] + ")")
        grid_checkoutmember[1,2] = widgets.HBox ([pet1_label])
        

      # SQL: get Member's pet names
        c.execute ("SELECT PET_NAME, PET_ID FROM PET WHERE CUSTOMER_CUS_ID = '{}' AND PET_SIZE = '{}' AND PET_SPECIES = '{}'".format(member_id, pet_sizes_list[0], pet_type))

        member_pet_list = ["New Pet"]
        member_pet_list_id = ["New Pet"]

        for row in c:
          member_pet_list.append(row[0])
          member_pet_list_id.append(row[1])
        
        global pet1_dictionary
        pet1_dictionary = dict(zip(member_pet_list,member_pet_list_id))           # PET 1 Dictionary

      # dropdown to choose pet
        pet1_dropdown = widgets.Dropdown(
          options = member_pet_list,
          description = 'Choose Pet:',
          style = {'description_width': '120px'},
          value = None
          )

        grid_checkoutmember[2,2] = widgets.HBox ([pet1_dropdown])

      # textbox if dropdown value is "New Pet"
        def new_pet_dropdown(b):
          global pet1_name
          if pet1_dropdown.value == "New Pet":
            new_pet1_name = widgets.Text(   
                value = "",                                             
                description='New Pet Name:',
                style = {'description_width': '120px'},
                layout = credentials_layout
            )
            grid_checkoutmember[3,2] = widgets.HBox ([new_pet1_name])

            pet1_name = ""

            def get_newpet1_name(b):
              global pet1_name
              pet1_name = new_pet1_name.value
            new_pet1_name.observe(get_newpet1_name, names='value')

          else:
            grid_checkoutmember[3,2] = widgets.HBox ([])

        pet1_dropdown.observe(new_pet_dropdown, names='value')

      ############################ Pet 2 Details ##############################      
      if no_of_pets >= 2:
        pet2_label = Label (pet_type + " 2 (" + pet_sizes_list[1] + ")")
        grid_checkoutmember[4,2] = widgets.HBox ([pet2_label])
        

      # SQL: get Member's pet names
        c.execute ("SELECT PET_NAME, PET_ID FROM PET WHERE CUSTOMER_CUS_ID = '{}' AND PET_SIZE = '{}' AND PET_SPECIES = '{}'".format(member_id, pet_sizes_list[1], pet_type))

        member_pet_list2 = ["New Pet"]
        member_pet_list_id2 = ["New Pet"]

        for row in c:
          member_pet_list2.append(row[0])
          member_pet_list_id2.append(row[1])
        
        global pet2_dictionary
        pet2_dictionary = dict(zip(member_pet_list2,member_pet_list_id2))        # PET 2 Dictionary
        
      
      # dropdown to choose pet
        pet2_dropdown = widgets.Dropdown(
          options = member_pet_list2,
          description = 'Choose Pet:',
          style = {'description_width': '120px'},
          value = None
          )

        grid_checkoutmember[5,2] = widgets.HBox ([pet2_dropdown])

      # textbox if dropdown value is "New Pet"
        def new_pet_dropdown2(b):
          global pet2_name
          if pet2_dropdown.value == "New Pet":
            new_pet2_name = widgets.Text(        
                value = "",                                        
                description='New Pet Name:',
                style = {'description_width': '120px'},
                layout = credentials_layout
            )
            grid_checkoutmember[6,2] = widgets.HBox ([new_pet2_name])

            pet2_name = ""

            def get_newpet2_name(b):
              global pet2_name
              pet2_name = new_pet2_name.value
            new_pet2_name.observe(get_newpet2_name, names='value')

          else:
            grid_checkoutmember[6,2] = widgets.HBox ([])
        
        pet2_dropdown.observe(new_pet_dropdown2, names='value')

      ############################ Pet 3 Details ##############################
      if no_of_pets >= 3:
        pet3_label = Label (pet_type + " 3 (" + pet_sizes_list[2] + ")")
        grid_checkoutmember[7,2] = widgets.HBox ([pet3_label])
        

      # SQL: get Member's pet names
        c.execute ("SELECT PET_NAME, PET_ID FROM PET WHERE CUSTOMER_CUS_ID = '{}' AND PET_SIZE = '{}' AND PET_SPECIES = '{}'".format(member_id, pet_sizes_list[2], pet_type))

        member_pet_list3 = ["New Pet"]
        member_pet_list_id3 = ["New Pet"]

        for row in c:
          member_pet_list3.append(row[0])
          member_pet_list_id3.append(row[1])
        
        global pet3_dictionary
        pet3_dictionary = dict(zip(member_pet_list3,member_pet_list_id3))        # PET 3 Dictionary

      
      # dropdown to choose pet
        pet3_dropdown = widgets.Dropdown(
          options = member_pet_list3,
          description = 'Choose Pet:',
          style = {'description_width': '120px'},
          value = None
          )

        grid_checkoutmember[8,2] = widgets.HBox ([pet3_dropdown])

      # textbox if dropdown value is "New Pet"
        def new_pet_dropdown3(b):
          global pet3_name
          if pet3_dropdown.value == "New Pet":
            new_pet3_name = widgets.Text(    
                value = "",                                            
                description='New Pet Name:',
                style = {'description_width': '120px'}
                
            )
            grid_checkoutmember[9,2] = widgets.HBox ([new_pet3_name])

            pet3_name = ""
                        
            def get_newpet3_name(b):
              global pet3_name
              pet3_name = new_pet3_name.value
            new_pet3_name.observe(get_newpet3_name, names='value')

          else:
            grid_checkoutmember[9,2] = widgets.HBox ([])

        pet3_dropdown.observe(new_pet_dropdown3, names ='value')

    ############################ Special Request ###############################
    special_request_label = Label('Special Request:')
    special_request = widgets.Textarea(layout=widgets.Layout(height="90%", width="80%"),
                                       placeholder = 'Insert special request here',
                                       )

    ######################### Next: Review Button ##############################
    next_reviewmember = widgets.Button(description = "Next: Review", layout=button_center)
    back_checkout = widgets.Button(description = "Resubmit", layout=button_center)

    ############################ Confirm Button ################################
    discount_button = widgets.Button(                                                
        description = 'Apply Discount',
        button_style = '',
        layout=button_center
    ) 
   
    ############################ Discount Code #################################
    discount_label = Label("Do you have a discount code?")
    discount_code = widgets.Text(                                                
        value = '',
        description='Discount Code:',
        style = {'description_width': '89px'}
        
    )
    
    ############################ Check Discount Code ###########################
    global dis_id
    global dis_percent
    global confirmed_discount_id
    confirmed_discount_id = 0
    dis_id = ""
    dis_percent = 1  

    def on_discount_button_clicked(b):
      global dis_percent
      global dis_id
      global final_subtotal
      dis_id = ""
      dis_percent = 1

      if discount_code.value != "":
        c.execute("SELECT DIS_ID, DIS_CODE, DIS_PERCENTAGE, DIS_EXPIRY_DATE, DIS_START_DATE FROM DISCOUNT_CODE WHERE DIS_CODE = '{}'".format(discount_code.value))

        for row in c:
          dis_id = row[0]
          dis_percent = row[2]
          dis_expiry_date = row[3]
          dis_start_date = row[4]
        
        if dis_id != "":
          if date.today() >= datetime.date(dis_expiry_date):
            grid_checkoutmember[16,0:2] = widgets.HBox ([Label ("")]) 
            grid_checkoutmember[17,0:2] = widgets.HBox ([Label ("Subtotal: $" + str(final_subtotal))]) 
            grid_checkoutmember[19,0:2] = widgets.HBox ([Label ("Coupon code is expired")])
            dis_percent = 1
          elif date.today() <= datetime.date(dis_start_date):
            grid_checkoutmember[16,0:2] = widgets.HBox ([Label ("")])
            grid_checkoutmember[17,0:2] = widgets.HBox ([Label ("Subtotal: $" + str(final_subtotal))]) 
            grid_checkoutmember[19,0:2] = widgets.HBox ([Label ("You've applied the coupon code too soon, come back later!")])
            dis_percent = 1
          else:
            final_subtotal = round(dis_percent * temporary_subtotal,2)
            grid_checkoutmember[15,0:2] = widgets.HBox ([Label ("$" + str(confirmed_type_price) + " HKD x " + str(nights) + " nights: $" + str(temporary_subtotal))])
            grid_checkoutmember[16,0:2] = widgets.HBox ([Label (str(round(1-dis_percent,2)*100) + "% Discount: $" + str(round(final_subtotal - temporary_subtotal,2)))]) 
            grid_checkoutmember[17,0:2] = widgets.HBox ([Label ("Subtotal: $" + str(final_subtotal))])
            grid_checkoutmember[19,0:2] = widgets.HBox ([Label ("")])

            global confirmed_discount_id
            confirmed_discount_id = dis_id                                          # DISCOUNT_CODE_DIS_ID        

        else:
          final_subtotal = temporary_subtotal
          grid_checkoutmember[16,0:2] = widgets.HBox ([Label ("")]) 
          grid_checkoutmember[17,0:2] = widgets.HBox ([Label ("Subtotal: $" + str(final_subtotal))])
          grid_checkoutmember[19,0:2] = widgets.HBox ([Label ("Coupon code is not valid")])
          dis_percent = 1
      
      # if they press the button, but the textbox empty
      else:
        final_subtotal = temporary_subtotal
        grid_checkoutmember[16,0:2] = widgets.HBox ([Label ("")])
        grid_checkoutmember[17,0:2] = widgets.HBox ([Label ("Subtotal: $" + str(final_subtotal))])
        grid_checkoutmember[19,0:2] = widgets.HBox ([Label ("")]) 
        dis_percent = 1

    discount_button.on_click(on_discount_button_clicked)

    ############################### Next: Payment ##############################
    def on_next_reviewmember_clicked(b):
      flag3 = False

      if mem_status == True:
        global confirmed_stars  
        global confirmed_stars_reduced
        global confirmed_rewards
        global confirmed_rewards_qty

        if rewards_dropdown.value != None:
          confirmed_rewards_qty = rewards_qty_intbox.value                         # CLA_QUANTITY 

        confirmed_rewards = rewards_dropdown.value                               # REWARD_NAME
        confirmed_stars = order_stars                                            # RES_STARS
      
        # if the customer is a member and the member chooses a reward, save the reward price and quantity chosen
        if rewards_dropdown.value != None:
          rew_price = rew_dict[rewards_dropdown.value]
          rew_qty_chosen = rewards_qty_intbox.value

          confirmed_stars_reduced = rew_price*rew_qty_chosen

          # if the member does not have enough stars to claim the rewards
          if (rew_price*rew_qty_chosen) > member_stars:
            grid_checkoutmember[21,2] = widgets.HBox([Label ("Sorry, you do not have enough stars!")])
            flag3 = True
        
      if no_of_pets >= 1:

        if pet1_dropdown.value == None: 
          grid_checkoutmember[21,2] = widgets.HBox ([Label("Please choose Pet 1")])
          flag3 = True

        if pet1_dropdown.value == "New Pet":
          if pet1_name == "":
            grid_checkoutmember[21,2] = widgets.HBox ([Label("Please fill in Pet 1 name")])
            flag3 = True

      if no_of_pets >= 2:
        if pet2_dropdown.value == None: 
          grid_checkoutmember[21,2] = widgets.HBox ([Label("Please choose Pet 2")])
          flag3 = True

        if pet2_dropdown.value == "New Pet":
          if pet2_name == "":
            grid_checkoutmember[21,2] = widgets.HBox ([Label("Please fill in Pet 2 name")])
            flag3 = True

      if no_of_pets >= 3:
        if pet3_dropdown.value == None: 
          grid_checkoutmember[21,2] = widgets.HBox ([Label("Please choose Pet 3")])
          flag3 = True

        if pet3_dropdown.value == "New Pet":
          if pet3_name == "":
            grid_checkoutmember[21,2] = widgets.HBox ([Label("Please fill in Pet 3 name")])
            flag3 = True

      if not flag3:

        flag3 = False
        grid_checkoutmember[21,2] = widgets.HBox ([Label("")])

        # save special request as variable
        global confirmed_special_request
        confirmed_special_request = special_request.value                        # RES_SPEC_REQ

        # save net bill as variable
        global confirmed_net_bill 
        confirmed_net_bill = final_subtotal                                      # RES_NET_BILL

        global confirmed_gross_bill
        confirmed_gross_bill = temporary_subtotal                                # RES_GROSS_BILL

        # save pet names
        if no_of_pets >= 1:
          global confirmed_pet1_name
          global confirmed_pet1_size
          confirmed_pet1_size = pet_sizes_list[0]                                # PET_SIZE (1)

          if pet1_dropdown.value == "New Pet":
            confirmed_pet1_name = pet1_name                                      # PET_NAME (1)
          else:
            confirmed_pet1_name = pet1_dropdown.value                   

        if no_of_pets >= 2:
          global confirmed_pet2_name
          global confirmed_pet2_size
          confirmed_pet2_size = pet_sizes_list[1]                                # PET_SIZE (2)

          if pet2_dropdown.value == "New Pet":
            confirmed_pet2_name = pet2_name                                      # PET_NAME (2)
          else:
            confirmed_pet2_name = pet2_dropdown.value

        if no_of_pets >= 3:
          global confirmed_pet3_name
          global confirmed_pet3_size
          confirmed_pet3_size = pet_sizes_list[2]                                # PET_SIZE (3)

          if pet3_dropdown.value == "New Pet":
            confirmed_pet3_name = pet3_name                                      # PET_NAME (3)
          else:
            confirmed_pet3_name = pet3_dropdown.value

        # move to the next page
        next_reviewmember.on_click(show_payment_preview)
        showPage("Payment")
        
    next_reviewmember.on_click(on_next_reviewmember_clicked)

    ############################ GRID ##############################
    grid_checkoutmember = GridspecLayout(23, 3, height='1000px', width='100%', grid_gap='1px')
    grid_checkoutmember[11,2] = widgets.HBox ([special_request_label])
    grid_checkoutmember[12:14,2] = widgets.HBox ([special_request])
    grid_checkoutmember[20,2] = widgets.HBox ([next_reviewmember])

    grid_checkoutmember[0,0] = widgets.HBox([back_checkout])
    grid_checkoutmember[13,0] = widgets.HBox ([discount_code])
    grid_checkoutmember[14,0] = widgets.HBox ([discount_button])

   
    display(grid_checkoutmember)

  #######################################################
  ################## GO TO: Payment Tab  ################
  #######################################################

  with tb_all.output_to('Payment'):
    ########################### Back Button ##############################
    back_payment = widgets.Button(description = "Back: Checkout", layout=button_center)

    def on_back_payment_clicked(b):
      grid_checkoutmember[3,2] = widgets.HBox ([Label("")])
      grid_checkoutmember[6,2] = widgets.HBox ([Label("")])
      grid_checkoutmember[9,2] = widgets.HBox ([Label("")])
      showPage("Checkout")
    
    back_payment.on_click(on_back_payment_clicked)

    ############################ Update Details ##############################
    def show_payment_preview(b):
      # special request
      grid_payment[11,0:2] = widgets.HBox ([Label("Special Request: " + confirmed_special_request)])

      if mem_status == True:
        if rewards_dropdown.value != None:
          grid_payment[12,0:2] = widgets.HBox ([Label("Claimed Rewards: " + str(confirmed_rewards_qty) + " " + confirmed_rewards)])
          grid_payment[13,0:2] = widgets.HBox ([Label("Rewards Cost: " + str(confirmed_stars_reduced) + "☆" )])
          grid_payment[14,0:2] = widgets.HBox ([Label("Stars Remaining: " + str(member_stars - confirmed_stars_reduced) + "☆" )])
        
        grid_payment[4,2] = widgets.HBox ([Label ("Stars Earned After Checkout: " + str(confirmed_stars))]) 
      
      if no_of_pets >= 1:
        grid_payment[15,0:2] = widgets.HBox ([Label(pet_type + " 1 (" + confirmed_pet1_size + "): " + confirmed_pet1_name)])            

      if no_of_pets >= 2:
        grid_payment[16,0:2] = widgets.HBox ([Label(pet_type + " 2 (" + confirmed_pet2_size + "): " + confirmed_pet2_name)])   

      if no_of_pets >= 3:
        grid_payment[17,0:2] = widgets.HBox ([Label(pet_type + " 3 (" + confirmed_pet2_size + "): " + confirmed_pet3_name)])   
      
      grid_payment[1,2] = widgets.HBox ([Label ("$" + str(confirmed_type_price) + " HKD x " + str(nights) + " nights: $" + str(confirmed_gross_bill))])

      if discount_code.value != "":
        grid_payment[2,2] = widgets.HBox ([Label (str(round(1-dis_percent,2)*100) + "% Discount: $" + str(round(confirmed_net_bill - confirmed_gross_bill,2)))]) 

      grid_payment[3,2] = widgets.HBox ([Label ("Subtotal: $" + str(final_subtotal))])


    ############################ Pay Button ##############################
    pay_button = widgets.Button(description = "Pay", layout=button_center)

    def on_pay_button_clicked(b):

      ################ Reservation ID ###############

      c.execute("SELECT RES_ID FROM RESERVATION ORDER BY (RES_ID)")
      checker = 0

      for row in c:
        res_id_no = int(row[0][3:]) + 1

        if checker < res_id_no:
          checker = res_id_no

      str_res_id_no = str(checker)
      confirmed_res_id = "RES" + str_res_id_no                                    # RES_ID

      ################ SQL: INSERT INTO RESERVATION ###############
      occu_status = "Yet to Check In"

      if mem_status == False:
        if confirmed_discount_id == 0:
          c.execute (
            """INSERT INTO RESERVATION (RES_ID, RES_CHECK_IN, RES_CHECK_OUT, RES_GROSS_BILL, RES_NET_BILL, RES_STARS, RES_SPEC_REQ, RES_NIGHTS, CUSTOMER_CUS_ID, RES_OCCU_STATUS)
            VALUES ('{}', TO_DATE('{}', 'YYYY-MM-DD'), TO_DATE('{}', 'YYYY-MM-DD'), {}, {}, {}, '{}', {}, '{}', 'Yet to Check In')"""
            .format(confirmed_res_id, check_in_datepicker.value.strftime("%Y-%m-%d"), check_out_datepicker.value.strftime("%Y-%m-%d"), confirmed_gross_bill, confirmed_net_bill, 0, confirmed_special_request, nights, confirmed_id)
        ) 

          conn.commit()

        else:

          c.execute (
            """INSERT INTO RESERVATION (RES_ID, RES_CHECK_IN, RES_CHECK_OUT, RES_GROSS_BILL, RES_NET_BILL, RES_STARS, 
            RES_SPEC_REQ, RES_NIGHTS, CUSTOMER_CUS_ID, DISCOUNT_CODE_DIS_ID, RES_OCCU_STATUS)
            VALUES ('{}', TO_DATE('{}', 'YYYY-MM-DD'), TO_DATE('{}', 'YYYY-MM-DD'), {}, {}, {}, '{}', {}, '{}', {}, '{}')"""
            .format(confirmed_res_id, check_in_datepicker.value.strftime("%Y-%m-%d"), check_out_datepicker.value.strftime("%Y-%m-%d"), confirmed_gross_bill, confirmed_net_bill, 0, confirmed_special_request, nights, confirmed_id, confirmed_discount_id, occu_status)) 

          conn.commit()

      else:
        if confirmed_discount_id == 0:
          c.execute (
            """INSERT INTO RESERVATION (RES_ID, RES_CHECK_IN, RES_CHECK_OUT, RES_GROSS_BILL, RES_NET_BILL, RES_STARS, RES_SPEC_REQ, RES_NIGHTS, CUSTOMER_CUS_ID, RES_OCCU_STATUS)
            VALUES ('{}', TO_DATE('{}', 'YYYY-MM-DD'), TO_DATE('{}', 'YYYY-MM-DD'), {}, {}, {}, '{}', {}, '{}', 'Yet to Check In')"""
            .format(confirmed_res_id, check_in_datepicker.value.strftime("%Y-%m-%d"), check_out_datepicker.value.strftime("%Y-%m-%d"), confirmed_gross_bill, confirmed_net_bill, confirmed_stars, confirmed_special_request, nights, confirmed_id)
        ) 

          conn.commit()

        else:

          c.execute (
            """INSERT INTO RESERVATION (RES_ID, RES_CHECK_IN, RES_CHECK_OUT, RES_GROSS_BILL, RES_NET_BILL, RES_STARS, 
            RES_SPEC_REQ, RES_NIGHTS, CUSTOMER_CUS_ID, DISCOUNT_CODE_DIS_ID, RES_OCCU_STATUS)
            VALUES ('{}', TO_DATE('{}', 'YYYY-MM-DD'), TO_DATE('{}', 'YYYY-MM-DD'), {}, {}, {}, '{}', {}, '{}', {}, '{}')"""
            .format(confirmed_res_id, check_in_datepicker.value.strftime("%Y-%m-%d"), check_out_datepicker.value.strftime("%Y-%m-%d"), confirmed_gross_bill, confirmed_net_bill, confirmed_stars, confirmed_special_request, nights, confirmed_id, confirmed_discount_id, occu_status)) 

          conn.commit()
        

      ################## SQL: INSERT INTO CONTAINS ################
      c.execute("INSERT INTO CONTAINS VALUES ('{}', '{}')".format(confirmed_res_id, confirmed_room_id))

      conn.commit()

      #################### Pet 1 ID #####################
      if no_of_pets >= 1:
        if pet1_dropdown.value == "New Pet":
          pet1_id = 0
          c.execute("SELECT PET_ID FROM PET ORDER BY (PET_ID)")
          
          for row in c:
            pet_id_no = row[0] + 1
            if pet1_id < pet_id_no:
              pet1_id = pet_id_no                                                 # PET_ID (1) if New Pet

          c.execute(
              """ INSERT INTO PET (PET_ID, PET_NAME, PET_SPECIES, PET_SIZE, CUSTOMER_CUS_ID) VALUES ('{}', '{}', '{}', '{}', '{}')"""
              .format(pet1_id, confirmed_pet1_name, pet_type, confirmed_pet1_size, confirmed_id))
          conn.commit()

        else:
          pet1_id = pet1_dictionary[confirmed_pet1_name]                         # PET_ID (1) if Existing Pet
        
        ################## SQL: INSERT INTO IS_INCLUDED (Pet 1) ################
        c.execute("INSERT INTO IS_INCLUDED VALUES ('{}', '{}')".format(pet1_id, confirmed_res_id))

        conn.commit()
        
      #################### Pet 2 ID #####################
      if no_of_pets >= 2:
        if pet2_dropdown.value == "New Pet":
          pet2_id = 0
          c.execute("SELECT PET_ID FROM PET ORDER BY (PET_ID)")
          
          for row in c:
            pet_id_no = row[0] + 1
            if pet2_id < pet_id_no:
              pet2_id = pet_id_no                                                 # PET_ID (2) if New Pet

          c.execute(
              """ INSERT INTO PET (PET_ID, PET_NAME, PET_SPECIES, PET_SIZE, CUSTOMER_CUS_ID) VALUES ('{}', '{}', '{}', '{}', '{}')"""
              .format(pet2_id, confirmed_pet2_name, pet_type, confirmed_pet2_size, confirmed_id))
          conn.commit()

        else:
          pet2_id = pet2_dictionary[confirmed_pet2_name]                         # PET_ID (2) if Existing Pet

        ################## SQL: INSERT INTO IS_INCLUDED (Pet 2) ################
        c.execute("INSERT INTO IS_INCLUDED VALUES ('{}', '{}')".format(pet2_id, confirmed_res_id))

        conn.commit()
      
      #################### Pet 3 ID #####################
      if no_of_pets >= 3:
        if pet3_dropdown.value == "New Pet":
          pet3_id = 0
          c.execute("SELECT PET_ID FROM PET ORDER BY (PET_ID)")
          
          for row in c:
            pet_id_no = row[0] + 1
            if pet3_id < pet_id_no:
              pet3_id = pet_id_no                                                 # PET_ID (3) if New Pet

          c.execute(
              """ INSERT INTO PET (PET_ID, PET_NAME, PET_SPECIES, PET_SIZE, CUSTOMER_CUS_ID) VALUES ('{}', '{}', '{}', '{}', '{}')"""
              .format(pet3_id, confirmed_pet3_name, pet_type, confirmed_pet3_size, confirmed_id))
          conn.commit()
        else:
          pet3_id = pet3_dictionary[confirmed_pet3_name]                          # PET_ID (3) if Existing Pet

        ################## SQL: INSERT INTO IS_INCLUDED (Pet 3) ################
        c.execute("INSERT INTO IS_INCLUDED VALUES ('{}', '{}')".format(pet3_id, confirmed_res_id))

        conn.commit()

        # To Integrate Start: Raphael
        if mem_status == True:
          print(rewards_dropdown.value)
          if rewards_dropdown.value != None:
            ################## SQL: INSERT INTO REWARDS_CLAIMED ################      
            ################ Generate CLA_ID ###############
            c.execute("SELECT CLA_ID FROM REWARDS_CLAIMED ORDER BY (CLA_ID)")
            checker = 0

            for row in c:
              cla_id_no = int(row[0][3:]) + 1

              if checker < cla_id_no:
                checker = cla_id_no

            str_cla_id_no = str(checker)
            confirmed_cla_id = "CLA" + str_cla_id_no                                 # CLA_ID

            ################ Get REWARD_ID ###############
            c.execute("SELECT REWARD_ID FROM REWARD WHERE REWARD_NAME = '{}'".format(confirmed_rewards))

            for row in c:
              confirmed_reward_id = row[0]                                           # REWARD_ID

            ################ INSERT ###############
            c.execute(
                """ INSERT INTO REWARDS_CLAIMED VALUES ('{}', {}, '{}', '{}')"""
                .format(confirmed_cla_id,confirmed_rewards_qty,confirmed_reward_id,confirmed_res_id))
          
            conn.commit()

            ################## SQL: UPDATE REWARD (Quantity) ################ Raphael
            c.execute("SELECT REWARD_QUANTITY FROM REWARD WHERE REWARD_NAME = '{}'".format (rewards_dropdown.value))
            for row in c:
              current_rew_qty = row[0]

            c.execute(
                """ UPDATE REWARD SET REWARD_QUANTITY = {} WHERE REWARD_ID = '{}' """
                .format(current_rew_qty - confirmed_rewards_qty,confirmed_reward_id))
          
            conn.commit()

            ################## SQL: UPDATE CUSTOMER (Stars) ################ Raphael
            c.execute(
                """ UPDATE CUSTOMER SET CUS_STARS = {} WHERE CUS_ID = '{}' """
                .format(member_stars - confirmed_stars_reduced,confirmed_id))
          
            conn.commit()

        # To Integrate Finish: Raphael

      showPage("Success")
    
    pay_button.on_click(on_pay_button_clicked)

    ############################ GRID ##############################
    grid_payment = GridspecLayout(23, 3, height='1000px', width='100%', grid_gap='1px')
    grid_payment[0,0:2] = widgets.HBox ([back_payment])
    grid_payment[5,2] = widgets.HBox ([pay_button])
    

    display(grid_payment)

  #######################################################
  ################ GO TO: Size Chart Tab ################
  #######################################################
  with tb_all.output_to('Size Chart'):

    btn_size_back = widgets.Button(description="Back")
    def on_btn_size_back_clicked(b):
      showPage("Home Page")
    btn_size_back.on_click(on_btn_size_back_clicked)

    size_chart = None
    size_chart = GridspecLayout(4, 4, height='150px', width='800px', grid_gap='1px')

    ######################### LABELS ####################
    small = Label("Small Size")
    medium = Label("Medium Size")
    large = Label('Large Size')
    small_desc = Label("Size: <= 40 cm cm ")
    medium_desc = Label("Size: <40 - 50 cm")
    large_desc = Label("Size: 50 - 60 cm")
    small_example = Label("Example: Poodle")
    medium_example = Label("Example: Golden Retriever")
    large_example = Label("Example: Great Dane")
      
      
      
    ######################## GRID #######################
    size_chart[0,0] = small
    size_chart[0,1] = medium
    size_chart[0,2] = large
    size_chart[1,0] = small_desc
    size_chart[1,1] = medium_desc
    size_chart[1,2] = large_desc
    size_chart[2,0] = small_example
    size_chart[2,1] = medium_example
    size_chart[2,2] = large_example
    size_chart[3,0] = widgets.HBox([btn_size_back])
    display(size_chart)

  #######################################################
  ################ GO TO: Booking Preview Tab ################
  #######################################################
  with tb_all.output_to("Booking Preview"):
    ############################ Formatting ##############################
    button_center = widgets.Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    width='30%')

    credentials_layout = widgets.Layout(width='400px')

    ############################ Back Button ##############################
    back_main = widgets.Button(description = "Back: Home", layout=button_center)

    def on_back_main_clicked(b):
      clear_output()
      start()
    
    back_main.on_click(on_back_main_clicked)

    ############################ Login ##############################
    login_label2 = Label("Login")
    

    member_email2 = widgets.Text(                                                
        value = '',
        placeholder='Insert email',
        description='Email :',
        layout = credentials_layout
    )
    member_password2 = widgets.Password(                                                                            
        value = '',
        placeholder = 'Enter password',
        description = 'Password :',
        layout = credentials_layout
    )
    login_button2 = widgets.Button(                                                
        description = 'Login',
        button_style = '',
        layout=button_center
    )
    def login_button2_clicked(i):
      global confirmed_name2
      global confirmed_id2
      global res_dropdown

      input = "SELECT CUS_EMAIL, CUS_PASSWORD FROM CUSTOMER WHERE CUS_EMAIL = '" + member_email2.value + "' AND CUS_PASSWORD = '" + member_password2.value + "'"
      validation = False
      for row in c.execute(input):
        validation = True

      if len(member_email2.value) == 0 and len(member_password2.value) == 0:
        grid_booking[3,0] = Label("Please fill in email and password")
      
      elif len(member_email2.value) == 0 and len(member_password2.value) != 0:
        grid_booking[3,0] = Label("Please fill in email")
      
      elif len(member_email2.value) != 0 and len(member_password2.value) == 0:
        grid_booking[3,0] = Label("Please fill in password")

      elif validation:
        grid_booking[3,0] = Label("")
        c.execute ("SELECT CUS_NAME, CUS_ID FROM CUSTOMER WHERE CUS_EMAIL = '{}' AND CUS_PASSWORD = '{}'".format(member_email2.value, member_password2.value))

        for row in c:
          confirmed_name1 = row[0]
          confirmed_id2 = row[1]                                                   

        c.execute ("SELECT RES_ID, CUS_NAME FROM RESERVATION FULL JOIN CUSTOMER ON customer.cus_id = reservation.customer_cus_id WHERE CUS_EMAIL = '{}' AND CUS_PASSWORD = '{}' ".format(member_email2.value, member_password2.value))
        res_id_list={}
        for row in c: 
                
          res_id_list[row[0]] = row[1]

        res_dropdown = widgets.Dropdown(options = res_id_list.keys(), description = "Your Booking: ",
                                                style = {'description_width': '120px'},
                                                value = None)

        grid_booking[4,0] = widgets.HBox([res_dropdown])
        grid_booking[4,1] = widgets.HBox([select_button])
        
      else:
        grid_booking[3,0] = Label("Incorrect email or password, please reenter")

    login_button2.on_click(login_button2_clicked)
    select_button = widgets.Button(description = "Select", layout=button_center)
    
    def on_select_button_clicked(b):
      global res_dropdown
      
      c.execute(        
        """
        SELECT CUS_NAME, RES_CHECK_IN, RES_CHECK_OUT, RES_NIGHTS, RES_SPEC_REQ, ROM_NO, TYP_NAME, RES_NET_BILL, RES_GROSS_BILL, TYP_SPECIES, REWARD_REWARD_ID, REWARD_NAME, REWARD_QUANTITY, CUS_MEMBERSHIP
        FROM RESERVATION FULL OUTER JOIN CUSTOMER ON RESERVATION.CUSTOMER_CUS_ID = CUSTOMER.CUS_ID 
        FULL OUTER JOIN CONTAINS ON RESERVATION.RES_ID = CONTAINS.RESERVATION_RES_ID
        FULL OUTER JOIN ROOM ON CONTAINS.ROOM_ROM_ID = ROOM.ROM_ID
        FULL OUTER JOIN ROOM_TYPE on room.room_type_typ_id = room_type.typ_id
        FULL OUTER JOIN REWARDS_CLAIMED on RESERVATION.RES_ID = REWARDS_CLAIMED.RESERVATION_RES_ID
        FULL OUTER JOIN REWARD on REWARDS_CLAIMED.REWARD_REWARD_ID = REWARD.REWARD_ID
        
        WHERE RESERVATION.RES_ID = '{}'
        """.format(res_dropdown.value)
        )
    
      conn.commit()
       
    
      for row in c: 
        CICO_customer_name_print = row[0] 
        CICO_check_in_date_print = row[1]
        CICO_check_out_date_print = row[2]
        CICO_nights_print = row[3]
        CICO_special_request_print = row[4]
        CICO_room_no_print = row[5]
        CICO_room_name = row[6]
        confirmed_net_bill = row[7]
        confirmed_gross_bill = row[8]
        type_species = row[9]
        reward = row[10]
        reward_name = row[11]
        reward_quantity = row[12]
        CICO_membership = row[13]
        
      

      CICO_customer_name_print_label = Label("Customer Name: "+str(CICO_customer_name_print))
      CICO_check_in_date_print_label = Label("Check In: "+str(CICO_check_in_date_print.strftime("%Y-%m-%d")))
      CICO_check_out_date_print_label = Label("Check Out: "+str(CICO_check_out_date_print.strftime("%Y-%m-%d")))
      CICO_nights_print_label = Label("Booked Nights: "+str(CICO_nights_print))
      CICO_special_request_print_label = Label("Special Requirements: "+str(CICO_special_request_print))
      CICO_room_name_label = Label("Room Name: "+str(CICO_room_name))

      
      
      grid_booking[5,0] = CICO_customer_name_print_label
      grid_booking[6,0] = CICO_room_name_label
      grid_booking[7,0] = CICO_check_in_date_print_label
      grid_booking[8,0] = CICO_check_out_date_print_label
      grid_booking[9,0] = CICO_nights_print_label
      grid_booking[10,0] = CICO_special_request_print_label

      petname_petsize_dict ={}
      petname_list=[]
      petsize_list=[]
      c.execute(        
              """SELECT PET_NAME, PET_SIZE  FROM RESERVATION FULL OUTER JOIN CUSTOMER ON RESERVATION.CUSTOMER_CUS_ID = CUSTOMER.CUS_ID 
              FULL OUTER JOIN CONTAINS ON RESERVATION.RES_ID = CONTAINS.RESERVATION_RES_ID
              FULL OUTER JOIN ROOM ON CONTAINS.ROOM_ROM_ID = ROOM.ROM_ID
              FULL OUTER JOIN ROOM_TYPE on room.room_type_typ_id = room_type.typ_id
              FULL OUTER JOIN is_included ON reservation.res_id = is_included.reservation_res_id 
              FULL OUTER JOIN PET ON pet.pet_id = is_included.pet_pet_id
              WHERE RESERVATION.RES_ID = '{}'""".format(res_dropdown.value))
      for row in c:
        petname_petsize_dict[row[0]] =row[1]

      no_of_pets=0
      for name,size in petname_petsize_dict.items():
        no_of_pets+=1
        petname_list.append(name)
        petsize_list.append(size)

     
      c.execute("""SELECT DIS_PERCENTAGE FROM RESERVATION 
          INNER JOIN DISCOUNT_CODE ON RESERVATION.DISCOUNT_CODE_DIS_ID =  DISCOUNT_CODE.DIS_ID
            WHERE RES_ID ='{}'""".format(res_dropdown.value))
      
      dis_percent_preview = ""
      for row in c:
        dis_percent_preview =row[0]
        
       
      if no_of_pets == 1:
        
        grid_booking[11,0:2] = widgets.HBox ([Label(type_species + " 1 (" + petsize_list[0] + "): " + petname_list[0])]) 
        grid_booking[12,0:2] = widgets.HBox([])
        grid_booking[13,0:2] = widgets.HBox([])
        if dis_percent_preview!= "":
          grid_booking[14,0:2] = widgets.HBox ([Label (str(round(1-dis_percent_preview,2)*100) + "% Discount: $" + str(round(confirmed_net_bill - confirmed_gross_bill,2)))]) 
        grid_booking[15,0:2] = widgets.HBox ([Label ("Subtotal: $" + str(confirmed_net_bill))])
        # if CICO_membership == "Yes":   
        #   grid_booking[16,0:2] = widgets.HBox ([Label ("Reward: " + str(reward_name) + " Quantity: " + str(reward_quantity))]) 

      elif no_of_pets ==2 :
        grid_booking[11,0:2] = widgets.HBox ([Label(type_species + " 1 (" + petsize_list[0] + "): " + petname_list[0])]) 
        grid_booking[12,0:2] = widgets.HBox ([Label(type_species + " 2 (" + petsize_list[1] + "): " + petname_list[1])]) 
        grid_booking[13,0:2] = widgets.HBox([])
        if dis_percent_preview!= "":
          grid_booking[14,0:2] = widgets.HBox ([Label (str(round(1-dis_percent_preview,2)*100) + "% Discount: $" + str(round(confirmed_net_bill - confirmed_gross_bill,2)))]) 
        grid_booking[15,0:2] = widgets.HBox ([Label ("Subtotal: $" + str(confirmed_net_bill))])
        # if CICO_membership == "Yes":   
        #   grid_booking[16,0:2] = widgets.HBox ([Label ("Reward: " + str(reward_name) + " Quantity: " + str(reward_quantity))]) 

      else :
        grid_booking[11,0:2] = widgets.HBox ([Label(type_species + " 1 (" + petsize_list[0] + "): " + petname_list[0])]) 
        grid_booking[12,0:2] = widgets.HBox ([Label(type_species + " 2 (" + petsize_list[1] + "): " + petname_list[1])])
        grid_booking[13,0:2] = widgets.HBox ([Label(type_species + " 3 (" + petsize_list[2] + "): " + petname_list[2])])
        if dis_percent_preview!= "":
          grid_booking[14,0:2] = widgets.HBox ([Label (str(round(1-dis_percent_preview,2)*100) + "% Discount: $" + str(round(confirmed_net_bill - confirmed_gross_bill,2)))]) 
        grid_booking[15,0:2] = widgets.HBox ([Label ("Subtotal: $" + str(confirmed_net_bill))])  
        # if CICO_membership == "Yes":   
        #   grid_booking[16,0:2] = widgets.HBox ([Label ("Reward: " + str(reward_name) + " Quantity: " + str(reward_quantity))]) 
     
    select_button.on_click(on_select_button_clicked)

    
    grid_booking = GridspecLayout(17, 4, height='1000px', width='100%', grid_gap='1px')
    grid_booking[0,0] = widgets.HBox([back_main])
    grid_booking[1,0] = widgets.HBox([member_email2])
    grid_booking[2,0] = widgets.HBox([member_password2])
    grid_booking[3,1] = widgets.HBox([login_button2])
    display(grid_booking)

    ############################ Formatting ##############################

  #######################################################
  ################ GO TO: Success Tab #################
  #######################################################
  with tb_all.output_to('Success'):
    success_grid = GridspecLayout(2, 1, height='300px', width='100%', grid_gap='1px')
    success_grid[0, 0] = widgets.HBox([Label("Your booking is confirmed!")])

    view_btn = widgets.Button (description = "View Bookings", disabled = False)
    success_grid[1, 0] = widgets.HBox([view_btn])

    def on_view_btn_clicked(b):
      showPage("Booking Preview")
    view_btn.on_click(on_view_btn_clicked)

    display(success_grid)

  showPage("Home Page")
start() 


    
      

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>